In [ ]:
import json
import os
from collections import defaultdict
import numpy as np
from torch.utils.data import Dataset
import torch



class FEMNIST(Dataset):
    """
    This dataset is derived from the Leaf repository
    (https://github.com/TalwalkarLab/leaf) pre-processing of the Extended MNIST
    dataset, grouping examples by writer. Details about Leaf were published in
    "LEAF: A Benchmark for Federated Settings" https://arxiv.org/abs/1812.01097.
    """

    def __init__(self, train=True, transform=None, target_transform=None, ):
        super(FEMNIST, self).__init__()
        self.transform = transform
        self.target_transform = target_transform
        self.train = train

        train_clients, train_groups, train_data_temp, test_data_temp = read_data("./data/femnist/train",
                                                                                 "./data/femnist/test")
        if self.train:
            self.dic_users = {}
            train_data_x = []
            train_data_y = []
            for i in range(len(train_clients)):
                if i == 100:
                    break
                self.dic_users[i] = set()
                l = len(train_data_x)
                cur_x = train_data_temp[train_clients[i]]['x']
                cur_y = train_data_temp[train_clients[i]]['y']
                for j in range(len(cur_x)):
                    self.dic_users[i].add(j + l)
                    train_data_x.append(np.array(cur_x[j]).reshape(28, 28))
                    train_data_y.append(cur_y[j])
            self.data = train_data_x
            self.label = train_data_y
        else:
            test_data_x = []
            test_data_y = []
            for i in range(len(train_clients)):
                cur_x = test_data_temp[train_clients[i]]['x']
                cur_y = test_data_temp[train_clients[i]]['y']
                for j in range(len(cur_x)):
                    test_data_x.append(np.array(cur_x[j]).reshape(28, 28))
                    test_data_y.append(cur_y[j])
            self.data = test_data_x
            self.label = test_data_y

    def __getitem__(self, index):
        img, target = self.data[index], self.label[index]
        img = np.array([img])
        # img = Image.fromarray(img, mode='L')
        # if self.transform is not None:
        #     img = self.transform(img)
        # if self.target_transform is not None:
        #     target = self.target_transform(target)
        return torch.from_numpy((0.5-img)/0.5).float(), target

    def __len__(self):
        return len(self.data)

    def get_client_dic(self):
        if self.train:
            return self.dic_users
        else:
            exit("The test dataset do not have dic_users!")
def batch_data(data, batch_size, seed):
    '''
    data is a dict := {'x': [numpy array], 'y': [numpy array]} (on one client)
    returns x, y, which are both numpy array of length: batch_size
    '''
    data_x = data['x']
    data_y = data['y']

    # randomly shuffle data
    np.random.seed(seed)
    rng_state = np.random.get_state()
    np.random.shuffle(data_x)
    np.random.set_state(rng_state)
    np.random.shuffle(data_y)

    # loop through mini-batches
    for i in range(0, len(data_x), batch_size):
        batched_x = data_x[i:i + batch_size]
        batched_y = data_y[i:i + batch_size]
        yield (batched_x, batched_y)


def read_dir(data_dir):
    clients = []
    groups = []
    data = defaultdict(lambda: None)

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith('.json')]
    for f in files:
        file_path = os.path.join(data_dir, f)
        with open(file_path, 'r') as inf:
            cdata = json.load(inf)
        clients.extend(cdata['users'])
        if 'hierarchies' in cdata:
            groups.extend(cdata['hierarchies'])
        data.update(cdata['user_data'])

    clients = list(sorted(data.keys()))
    return clients, groups, data



def read_data(train_data_dir, test_data_dir):
    '''parses data in given train and test data directories

    assumes:
    - the data in the input directories are .json files with
        keys 'users' and 'user_data'
    - the set of train set users is the same as the set of test set users

    Return:
        clients: list of client ids
        groups: list of group ids; empty list if none found
        train_data: dictionary of train data
        test_data: dictionary of test data
    '''
    train_clients, train_groups, train_data = read_dir(train_data_dir)
    test_clients, test_groups, test_data = read_dir(test_data_dir)

    assert train_clients == test_clients
    assert train_groups == test_groups

    return train_clients, train_groups, train_data, test_data





In [ ]:

# test = FEMNIST(train=True)
# x = test.get_client_dic()
# print(len(x))
# t = 0
# print(x[2])
# # for k in x[1]:
# #     t += 1
# #     data, label = test.__getitem__(k)
# #     print(t)

In [ ]:
import matplotlib
import argparse
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn # import modules

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models_v1.Update import LocalUpdate
from models_v1.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2, MobileNetV2, LeNet5, LeNet10, cnn_cifar10, CNN_FEMNIST
from models_v1.Fed import FedAvg
from models_v1.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models_v1.test import test_img

%load_ext autoreload
%autoreload 2
from torch.autograd import grad
parser = argparse.ArgumentParser(description='Deep Leakage from Gradients.')
parser.add_argument('--index', type=int, default="25",
                    help='the index for leaking images on CIFAR.')
parser.add_argument('--image', type=str,default="",
                    help='the path to customized image.')
args = parser.parse_args([])
from torch.autograd import grad

In [ ]:
class my_argument:
    epochs = 400    #"rounds of training"
    num_users = 100  # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=5 #"the number of local epochs: E"
    local_bs=10 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'ADAM'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='femnist' #, help="name of dataset")
    iid=0
    num_classes=62#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    model='cnn'
    q=20
    f_size=32
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cpu")
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

In [ ]:
from models_v1.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient, weight_vectorization_gen2
net_glob = LeNet10().to(args.device)  #(args=args).to(args.device)
w_glob=net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))
print(net_glob)

In [ ]:
net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
net_glob.eval()
w_glob = net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))
#print(net_glob)

In [ ]:
from collections import defaultdict

arch = 'ConvNet64'
arch='cnn'
arch='ResNet18'
num_images = 5
trained_model = False

In [ ]:
def phiQ(p,sc, q, w):
    w_cap = w #[:,0]
    #w_cap=[item[0] if isinstance(item,list) and len(item)>0 else item for item in w_cap]
    #w_cap=np.array(w_cap)
    #print("w_cap_first")
    #print(w_cap)
    w_cap= sc*w_cap
    v=np.floor(q*w_cap)
    one=np.ones(len(w))
    r=np.random.uniform(0,1,len(w_cap))
    temp=(one.T+np.sign(q*w_cap-v-r))*np.sign(q*w_cap-v-r)
    #j=np.sign(w_cap-v-r)
    #print("jjj")
    #print(j)
    #temp= (1/q)*(1/2)*temp
    temp=(1/2)*temp
    #print("temp")
    #print(temp)
    #w_cap= (1/q)*v + temp
    w_cap=(1/q)*(v+temp)
    w_cap=q*w_cap
    #w_cap=w_cap+ (1/2)*p*(-np.sign(w_cap)+one.T)*(-np.sign(w_cap))
    w_cap=w_cap+ (1/2)*(p-5)*(-np.sign(w_cap)+one.T)*(-np.sign(w_cap))
    #print("w_cap_last")
    #print(w_cap)
    del temp
    del one
    del v
    del r
    return w_cap
args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

# load dataset and split users
# if args.dataset == 'mnist':
#     trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
#     dataset_train = datasets.MNIST('/data/mnist/', train=True, download=True, transform=trans_mnist)
#     dataset_test = datasets.MNIST('/data/mnist/', train=False, download=True, transform=trans_mnist)
#     # sample users
#     if args.iid:
#         dict_users = mnist_iid(dataset_train, args.num_users)
#         print('iid dataset')
#     else:
#         dict_users = mnist_noniid(dataset_train, args.num_users)
#         print("non iid dataset")
# elif args.dataset == 'cifar':
#     trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#     dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
#     dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)
#     if args.iid:
#         dict_users = cifar_iid(dataset_train, args.num_users)
#     else:
#         exit('Error: only consider IID setting in CIFAR10')
# else:
#     exit('Error: unrecognized dataset')
#img_size = dataset_train[0][0].shape


In [ ]:
# test = FEMNIST(train=True)
# dict_users = test.get_client_dic()
# print(len(dict_users))
# t = 0

In [ ]:
# test = FEMNIST(train=False)
# dict_users = test.get_client_dic()
# print(len(dict_users))
# t = 0

In [ ]:
def toNumpyFlatArray(self):
        return self.flat


def updateFromNumpyFlatArray(self, arr):
    self.flat = arr
    start = 0
    new_glob = OrderedDict()
    for k in self.w_glob.keys():
        size = 1
        for dim in self.w_glob[k].shape:
            size *= dim
        shaped = np.reshape(arr[start : start + size].copy(), self.w_glob[k].shape)
        new_glob[k] = torch.from_numpy(shaped)
        start = start + size
    self.w_glob = new_glob
    self.net_glob.load_state_dict(self.w_glob)

In [ ]:
# load dataset and split users
if args.dataset == 'mnist':
    trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    dataset_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=trans_mnist)
    dataset_test = datasets.MNIST('./data/mnist/', train=False, download=True, transform=trans_mnist)
    # sample users
    if args.iid:
        dict_users = mnist_iid(dataset_train, args.num_users)
    else:
        dict_users = mnist_noniid(dataset_train, args.num_users)
elif args.dataset == 'cifar':
    trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
    dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)
    if args.iid:
        print("iid dataset")
        dict_users = cifar_iid(dataset_train, args.num_users)
    else:
        exit('Error: only consider IID setting in CIFAR10')
elif args.dataset == 'femnist':
    dataset_train = FEMNIST(train=True)
    dataset_test = FEMNIST(train=False)
    dict_users = dataset_train.get_client_dic()
    
else:
    exit('Error: unrecognized dataset')
img_size = dataset_train[0][0].shape

In [ ]:
print(len(dict_users[2]))

# 1. FedAvg with A=0

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        nn.init.xavier_uniform(m.weight.data, nn.init.calculate_gain('relu'))
        #nn.init.xavier_uniform(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
    if isinstance(m, torch.nn.Linear):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.BatchNorm2d):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)
        #conv1.bias.data.fill_(0.01)

In [ ]:

local_lr = 1e-4
local_steps = 5
use_updates = True

In [ ]:
from models_v1.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient
import numpy as np
import copy
import random
# build model
if args.model == 'cnn' and args.dataset == 'cifar':
    net_glob = cnn_cifar10(args=args).to(args.device)
elif args.model == 'cnn' and args.dataset == 'mnist':
    net_glob = CNNMnist(args=args).to(args.device)
elif args.model == 'cnn' and args.dataset == 'femnist':
    net_glob = CNN_FEMNIST(args=args).to(args.device)
elif args.model == "mobilenetv2":
    net_glob = MobileNetV2(args=args).to(args.device)
elif args.model == 'mlp':
    len_in = 1
    for x in img_size:
        len_in *= x
    net_glob = MLP(dim_in=len_in, dim_hidden=200, dim_out=args.num_classes).to(args.device)
else:
    exit('Error: unrecognized model')
print(net_glob)
net_glob.apply(weights_init)
net_glob.train()
#torch.save(net_glob.state_dict(),"model_acc_FedAvg.pt")

In [ ]:
w_glob = net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)
print(len(g))

In [ ]:
#net_glob=ResNet18()
#torch.save(net_glob.state_dict(),  "Model_acc_femnist.pt")

In [ ]:
from models_v1.Fed import FedAdd,FedSubstract,weight_vectorization_cifar,FedAvg_gradient, weight_vectorization_gen, weight_vectorization_gen2
#net_glob = LeNet10().to(args.device)
#net_glob.train()
args.lr=0.001
import torchvision.models as models
dev=torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#net_glob =models.resnet18(pretrained=False, progress=True)
net_glob=net_glob.to(dev)
net_glob.train()
# copy weights
#net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
#net_glob.eval()
w_glob = net_glob.state_dict()
w_glob=torch.load("Model_acc_femnist.pt")
net_glob.load_state_dict(w_glob)
net_glob=net_glob.to(dev)
net_glob.train()
w_glob = net_glob.state_dict()
# training
loss_train = []
loss_test_arr = []
acc_test_arr = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []

m_local=[]
d=11699132 #resnet18
d= 11173962
d= 11699132
d=11183582
d=62006
d=1068298
d=6577694
iter_no=500
avg=[]
error=[]
idxs_users=range(0,args.num_users)
    #print(len(idxs_users))
user_no=args.num_users
updated=[]
for user in idxs_users:
        #print(user)
    updated.append([])
model_diff=[]




#net_glob.zero_grad()
input_gradient=[]

user_no=args.num_users
select=round(0.9*user_no)


sparsity=0.005
K_local=round(sparsity*d)
#net_glob.zero_grad()
for iter in range(iter_no): #args.epochs
    print("iteration no.",iter)
    m_local=[]
    f=[]
    if iter>=40:
        args.r=0.00025
        T=[]
    w_locals, loss_locals,diff_locals,grad_locals = [], [],[],[]
    m = 10
    updated=[]
    model_diff=[]
    grad_vect=[]
    prev=[]
    error=[]
    grad_vect_quant=[]
    grad_vect_quant2=[]
    grad_vect_send=[]
    grad_vect_send2=[]
    grad_vect_quant=[]
    store_grad=[]
    location_local=[]
    np.random.seed(iter)
    rand=np.random.choice(idxs_users,select,replace=False)
    rand=np.sort(rand)
    #rand=idxs_users
    loss_train_user=[]
    for i in range(args.num_users):
        updated.append([])
        model_diff.append([])
        grad_vect.append([])
        prev.append([])
        grad_vect_send.append([])
        error.append(np.zeros(d))
        grad_vect_quant.append([])
    #idxs_users = np.random.choice(range(args.num_users), m, replace=False)

        #T=np.zeros(user_no)
    
    for user in rand: #idxs_users:
        #print(user)


        prev=copy.deepcopy(w_glob)
        local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[user])
        w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
        #target_loss, _, _ = loss_fn(net_glob(ground_truth[user]), labels[user])
        #print("success")
        #prev[user]=updated[user]
        loss_train_user.append(loss)
        model_diff=FedSubstract(w,prev)


        g, dim = weight_vectorization_gen(model_diff)
        g=g[:,0]# +error[user]
        location_local=np.random.choice(range(d),K_local,replace=False)
        mask=np.zeros(d)
        np.put(mask,location_local,1)
        grad_vect= np.multiply(g,mask)
        scale=1/(sparsity*0.9*args.num_users)
        grad_vect_quant=phiQ(np.power(2,args.f_size),scale,2**args.q,grad_vect)
        #error[user]=g-grad_vect
        grad_locals.append(grad_vect_quant)
        #del g
        print(user)
    grad_avg=sum(grad_locals)  #/len(grad_locals)
    grad_avg=np.nan_to_num(sum(grad_locals))
    loss_train.append(sum(loss_train_user)/len(loss_train_user))
    grad_avg_correct = np.zeros_like(grad_avg)
    #print("before modulo")
    #print(grad_avg)
    grad_avg= (grad_avg)%(np.power(2,args.f_size)-5)
    #print("after modulo")
    #print(grad_avg)
    p=np.power(2,args.f_size)-5
    for i in range(len(grad_avg)):
        if grad_avg[i]>=0 and grad_avg[i]<(p-1)/2:
                        # print("Valid")
            grad_avg_correct[i]=grad_avg[i]
            grad_avg_correct[i]=(1/(2**args.q))*grad_avg_correct[i]
            #grad_avg_correct[i]=(1/(2**args.q))*grad_avg_correct[i]
            continue
        elif grad_avg[i]>=(p-1)/2 and grad_avg[i]<p:
                        # print("Chenged")
            grad_avg_correct[i]=grad_avg[i]-p
            grad_avg_correct[i]=(1/(2**args.q))*grad_avg_correct[i]
    count=0
    w_glob_prev=copy.deepcopy(w_glob)
    flat=[]
    #conver
    for i in range(len(w_glob.keys())): # 4 layers in parameter
        flat.append([])

    for h in w_glob_prev.keys():
        s=list(w_glob[h].shape)
        if (len(s)==0):
            new=np.array(0)
            grad_avg_correct=np.delete(grad_avg_correct,np.s_[0])
        else:
            z=np.prod(list(w_glob[h].shape))
            flat[count]=grad_avg_correct[0:z] # taking out the vector for the specified layer
            grad_avg_correct=np.delete(grad_avg_correct,np.s_[0:z]) # deleting that vector from decoded after taking out

            new=flat[count].reshape(list(w_glob[h].shape)) #reshaping back to the marix

        w_glob[h]=torch.from_numpy(new) #converting the matrix to a tensor
            #print(w_glob[cluster_no][h].shape)
        count=count+1
    global_diff = w_glob
    #print(w_glob)
    w_glob=FedAdd(w_glob_prev,global_diff)
    # update global weights
    #global_diff = w_glob
    #print(w_glob)
    #w_glob=FedAdd(w_glob_prev,global_diff)


    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    del w_glob_prev
    del grad_locals
    del grad_avg
    del flat
    torch.cuda.empty_cache()

    # print loss
    #loss_avg = np.nan_to_num(sum(loss_locals) / len(loss_locals))

    #loss_train.append(float(loss_avg))

    acc_test, loss_test = test_img(net_glob, dataset_test, args)
    acc_test_arr.append(float(acc_test))
    loss_test_arr.append(loss_test)
    if iter % 1 ==0:
        #print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))

        print("accuracy array")
        print(acc_test_arr[iter])
        print("train loss")
        print(loss_train[iter])







In [ ]:
print(acc_test_arr)

In [ ]:
plt.plot(range(len(acc_test_arr)), acc_test_arr)
plt.show()